In [ ]:
# default_exp train

In [ ]:
# export
from grade_classif.models.plmodules import Normalizer, GradesClassifModel

# Train functions
> Functions that wrap the whole training processes

In [ ]:
# export
def train_normalizer(hparams):
    model = Normalizer(hparams)
    model.fit()
    return model

Trains a `Normalizer` unet with parameters defined in `hparams`. Parameters for this task can be found in `params.norm`.

In [ ]:
# export
def train_classifier(hparams):
    model = GradesClassifModel(hparams)
    model.fit()
    return model

Trains a `GradesClassifModel` for grade classifications with parameters defined in `hparams`. Parameters for this task can be found in `params.norm`.

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_train.ipynb.
Converted 02_predict.ipynb.
Converted 10_data_read.ipynb.
Converted 11_data_loaders.ipynb.
Converted 12_data_dataset.ipynb.
Converted 13_data_utils.ipynb.
Converted 14_data_transforms.ipynb.
Converted 20_models_pl_modules.ipynb.
Converted 21_models_modules.ipynb.
Converted 22_models_utils.ipynb.
Converted 23_models_hooks.ipynb.
Converted 24_models_metrics.ipynb.
Converted 80_params_defaults.ipynb.
Converted 81_params_norm.ipynb.
Converted 82_params_classif.ipynb.
Converted 83_params_predict.ipynb.
Converted 99_index.ipynb.
